# Linear Programming Model Analysis with Gurobi

This notebook presents the setup, solution, and sensitivity analysis of a linear programming problem using the Gurobi optimization library.

## Model Overview

The model, named 'modelo 3', aims to **maximize** an objective function subject to three linear constraints.

### Decision Variables

The model includes five continuous decision variables:
* `xt`
* `xa`
* `xg`
* `ca`
* `va`

### Objective Function

The objective is to maximize the following expression:
`Maximize: 450*xg + 120*xt + 22*va + 28*xa - 28*ca`

### Constraints

The problem is subject to the following linear constraints:
1.  `xt + xa + 0.1*xg <= 800`
2.  `2*xt + 3*xa + 0.05*xg <= 1000`
3.  `4*xa + ca - 5*xg - va == 0`

## Optimal Solution

After solving the model, the Gurobi optimizer found an **optimal objective value** of **2,480,000.0**.

The optimal values for the decision variables are:
* `xt = 0.0`
* `xa = 0.0`
* `xg = 8000.0`
* `ca = 40000.0`
* `va = 0.0`

This indicates that to achieve the maximum objective, the optimal strategy involves producing `xg` at its maximum level (8000.0 units) and adjusting `ca` accordingly, while `xt`, `xa`, and `va` are not utilized.

## Sensitivity Analysis

Sensitivity analysis provides insights into how changes in the model's parameters might affect the optimal solution.

### Reduced Costs

The reduced cost of a variable indicates how much its objective coefficient would need to change for that variable to become non-zero in the optimal solution.
* `xt`: -2980.0
* `xa`: -2960.0
* `xg`: 0.0
* `ca`: 0.0
* `va`: -6.0

Variables `xg` and `ca` have a reduced cost of 0, as they are part of the optimal basis. For `xt`, `xa`, and `va`, their objective coefficients would need to increase by at least their absolute reduced cost values for them to enter the optimal solution.

### Slack Variables

The slack value for a constraint indicates the unused capacity or the difference between the left-hand side and right-hand side of the constraint.
* `R0` (`xt + xa + 0.1*xg <= 800`): 0.0 (Binding constraint - all capacity utilized)
* `R1` (`2*xt + 3*xa + 0.05*xg <= 1000`): 600.0 (Non-binding constraint - 600 units of slack)
* `R2` (`4*xa + ca - 5*xg - va == 0`): 0.0 (Equality constraint - perfectly satisfied)

### Dual Prices (Shadow Prices)

The dual price for a constraint indicates the change in the optimal objective value for a one-unit increase in the right-hand side of that constraint.
* `R0`: 3100.0 (Increasing the RHS of `R0` by 1 unit would increase the objective by 3100.0, indicating it's a bottleneck resource).
* `R1`: 0.0 (Increasing the RHS of `R1` would not change the objective as there's already slack).
* `R2`: -28.0 (Increasing the RHS of `R2` by 1 unit would decrease the objective by 28.0).

### Variable Objective Coefficient Sensitivity

This shows the range within which each variable's objective coefficient can change without altering the current optimal basis.
* `xt`: (-inf, 3100.0]
* `xa`: (-inf, 2988.0]
* `xg`: [154.0, inf)
* `ca`: [-82.81, -22.0]
* `va`: (-inf, 28.0]

### Constraint RHS Sensitivity

This indicates the range within which the right-hand side of each constraint can change without altering the current optimal basis.
* `R0`: [0.0, 2000.0] (Current RHS: 800.0)
* `R1`: [400.0, inf) (Current RHS: 1000.0)
* `R2`: [-40000.0, inf) (Current RHS: 0.0)

In [1]:
import gurobipy as gp

In [2]:
model = gp.Model('modelo 3')

Restricted license - for non-production use only - expires 2024-10-28


In [3]:

xt = model.addVar(vtype=gp.GRB.CONTINUOUS,name='xt')
xa = model.addVar(vtype=gp.GRB.CONTINUOUS,name='xa')
xg = model.addVar(vtype=gp.GRB.CONTINUOUS,name='xg')
ca = model.addVar(vtype=gp.GRB.CONTINUOUS,name='ca')
va = model.addVar(vtype=gp.GRB.CONTINUOUS,name='va')

In [4]:
model.setObjective(450*xg+120*xt+22*va+28*xa - 28*ca, gp.GRB.MAXIMIZE)

In [5]:
model.addConstr(xt+xa+0.1*xg<=800)
model.addConstr(2*xt+3*xa+0.05*xg<=1000)
model.addConstr(4*xa+ca - 5*xg-va==0)

<gurobi.Constr *Awaiting Model Update*>

In [6]:
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x16cbc942
Coefficient statistics:
  Matrix range     [5e-02, 5e+00]
  Objective range  [2e+01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+02, 1e+03]
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.7400000e+32   4.487500e+30   7.740000e+02      0s
       3    2.4800000e+06   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.480000000e+06


In [7]:
model.ObjVal

2480000.0

In [8]:
model.getVars()

[<gurobi.Var xt (value 0.0)>,
 <gurobi.Var xa (value 0.0)>,
 <gurobi.Var xg (value 8000.0)>,
 <gurobi.Var ca (value 40000.0)>,
 <gurobi.Var va (value 0.0)>]

In [10]:
xt.x

0.0

In [11]:
xg.x

8000.0

In [12]:
model.status # 2: solucion optima, 4 no tiene solucion

2

In [13]:
xt.varName

'xt'

In [14]:
xt.RC

-2980.0

In [17]:
# costo reduccion
for v in model.getVars():
    print(v.varName,":",v.RC)

xt : -2980.0
xa : -2960.0
xg : 0.0
ca : 0.0
va : -6.0


In [20]:
# holguras
for r in model.getConstrs():
    print(str(r)[-3:-1], ":",r.slack)

R0 : 0.0
R1 : 600.0
R2 : 0.0


In [21]:
# precio dual
for r in model.getConstrs():
    print(str(r)[-3:-1], ":",r.pi)

R0 : 3100.0
R1 : 0.0
R2 : -28.0


In [24]:
# Sensibilidad varaibles
for v in model.getVars():
    print(v.varName,":",v.SAObjLow,v.SAObjup)

xt : -inf 3100.0
xa : -inf 2988.0
xg : 154.0 inf
ca : -82.81481481481481 -22.0
va : -inf 28.0


In [26]:
# Sensibilidad restricciones
for r in model.getConstrs():
    print(str(r)[-3:-1], ":",r.RHS, ",",r.SARHSLow,",",r.SARHSUP)

R0 : 800.0 , 0.0 , 2000.0
R1 : 1000.0 , 400.0 , inf
R2 : 0.0 , -40000.0 , inf
